<a href="https://colab.research.google.com/github/sayantanig/noisy_labels/blob/master/Fractional_Imputation_Vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy.random import default_rng

import pandas as pd

import torch
import torch.nn as nn

from random import sample
from random import random

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics

import matplotlib.pyplot as plt

import math

In [ ]:
# Global variables

trainX = 0
trainY = 0
n=500
num_labels = 2
numY = 2
delta = 0

In [ ]:
# Generating datasets

def generate(n):
  trainX = np.random.choice([1,2,3,4], n, p=[0.3,0.25,0.25,0.2])
  # trainX

  trainY=[]
  for i in trainX:
    if i==1:
      y1=np.random.choice([0,1], 1, p=[0.6,0.4])[0]
      y2_p=(0.8+y1)/2
    elif i==2:
      y1=np.random.choice([0,1], 1, p=[0.7,0.3])[0]
      y2_p=0.8*y1+0.1
    elif i==3:
      y1=np.random.choice([0,1], 1, p=[0.6,0.4])[0]
      y2_p=0.9-0.5*y1
    else:
      y1=np.random.choice([0,1], 1, p=[0.2,0.8])[0]
      y2_p=0.6
    y2=np.random.choice([0,1], 1, p=[1-y2_p,y2_p])[0]
    trainY.append([y1,y2])
  # trainY

  return (trainX, trainY)

In [ ]:
# Generate delta

def delta_func():
  delta=[[0]*numY for i in range(n)]
  
  for i in range(n):
    for j in range(numY):
      delta[i][0]=np.random.choice([0,1], 1, p=[0.2,0.8])[0]
      delta[i][1]=np.random.choice([0,1], 1, p=[0.3,0.7])[0]
  
  return delta

In [ ]:
# Full sample estimator

def fs_estimator():
  theta_fs = [0 for i in range(numY)]
  
  for j in range(numY):
    for i in trainY:
      theta_fs[j] += i[j]
    theta_fs[j]/= n

  # print("Full sample estimator: ",theta_fs)
  
  return theta_fs

In [ ]:
# Complete case estimator

def cc_estimator():
  theta_cc = [0 for i in range(numY)]
  n_y = [0 for i in range(numY)] 

  for j in range(numY):
    for i in range(n):
      theta_cc[j] += trainY[i][j] * delta[i][j]
      n_y[j] += delta[i][j]
    theta_cc[j] /= n_y[j]

  # print("Complete case estimator: ",theta_cc)

  return theta_cc

In [ ]:
def reduce_mult(a,b):
  # a:missing pattern index, b:label index
  num=0
  p=0
  while(a!=0):
    dgt_a=a%2
    a=a//2
    dgt_b=b%num_labels
    b=b//num_labels
    num+=dgt_a*dgt_b*pow(num_labels, p)
    p+=1
  return num

In [ ]:
def pi_plus_update(pi_est):
  pi_plus = [[0 for i in range(pow(num_labels, numY))] for j in range(pow(2, numY))]
  
  for i in range(pow(2,numY)):
    for j in range(pow(num_labels,numY)):
      pi_plus[i][reduce_mult(i,j)] += pi_est[j]
  
  return pi_plus

In [ ]:
def n_update(pi_est, pi_plus_est, n_plus):
  n_new = [[0 for i in range(pow(num_labels, numY))] for j in range(pow(2, numY))]

  for i in range(pow(2,numY)):
    for j in range(pow(num_labels,numY)):
      n_new[i][j]=n_plus[i][reduce_mult(i,j)]*(pi_est[j]/pi_plus_est[i][reduce_mult(i,j)])

  return n_new

In [ ]:
def pi_update(pi_est, n_est, n_curr):
  error_max=0
  pi_new = [0 for i in range(pow(num_labels, numY))]

  for i in range(pow(2, numY)):
    for j in range(pow(num_labels, numY)):
      pi_new[j]+=n_est[i][j]

  for j in range(pow(num_labels, numY)):
    pi_new[j] /= n_curr
    error_max=max(error_max, abs(pi_est[j]-pi_new[j]))

  return (pi_new, error_max)

In [ ]:
def em_alg(trainY_curr):
  
  n_curr = len(trainY_curr)

  # S = [[] for i in range(pow(2, numY))]
  # for i in range(n_curr):
  #   st = ""
  #   for j in range(numY):
  #     st += str(delta[i][j])
  #   S[int(st,2)].append(i)

# Probably useless block of code
  n_labels = [[0 for i in range(pow(num_labels, numY))] for j in range(pow(2, numY))]
  for i in range(n_curr):
    st_row = ""
    st_col = ""
    for j in range(numY):
      st_row += str(delta[i][j])
      st_col += str(trainY_curr[i][j])
    n_labels[int(st_row, 2)][int(st_col, num_labels)] += 1
#--------------------------------

  n_plus = [[0 for i in range(pow(num_labels, numY))] for j in range(pow(2, numY))]
  for i in range(n_curr):
    st_row = ""
    st_col = ""
    for j in range(numY):
      st_row += str(delta[i][j])
      st_col += str(trainY_curr[i][j]*delta[i][j])
    n_plus[int(st_row, 2)][int(st_col, num_labels)] += 1

  pi_est = [1/pow(num_labels, numY) for i in range(pow(num_labels, numY))]
  error_curr = 1
  stop_eps = 0.00001

  while(error_curr > stop_eps):
    pi_plus_est = pi_plus_update(pi_est)
    n_est =  n_update(pi_est, pi_plus_est, n_plus)
    (pi_est, error_curr) = pi_update(pi_est, n_est, n_curr)

  return pi_est

In [ ]:
# Fractional imputation estimator

def fi_estimator():
  trainY_arr = [[],[],[],[]]
  w_est = [0, 0, 0, 0]

  for i in range(n):
    trainY_arr[trainX[i]-1].append(trainY[i])
  # print(trainY_arr)
  for i in range(4):
    w_est[i] = em_alg(trainY_arr[i])
  # print(w_est)
  
  theta_fi = [0 for i in range(numY)]
  for z in range(numY):
    for i in range(n):
      w_curr=0
      for j in range(pow(num_labels, numY)):
        w_curr += w_est[trainX[i]-1][j] * ((j//pow(num_labels,numY-z-1))%num_labels) #extracting (z+1)th digit
      theta_fi[z] += delta[i][z]*trainY[i][z] + (1-delta[i][z])*w_curr
    theta_fi[z]/=n
  return theta_fi

In [ ]:
# theta_fs = [0 for i in range(numY)]
# theta_cc = [0 for i in range(numY)]
# theta_fi = [0 for i in range(numY)]

theta_fs, theta_cc, theta_fi = [], [], []

B=1000

# delta = delta_func()
# while (min(np.sum(np.array(delta), axis=0))==0):
#   delta = delta_func()
  
# (trainX, trainY) = generate(n)

for i in range(B):
  delta = delta_func()
  if min(np.sum(np.array(delta), axis=0))==0:
    continue
  
  (trainX, trainY) = generate(n)

  theta_fs.append(fs_estimator())
  theta_cc.append(cc_estimator())
  theta_fi.append(fi_estimator())

In [ ]:
fs_mean = np.mean(np.array(theta_fs), axis=0)
cc_mean = np.mean(np.array(theta_cc), axis=0)
fi_mean = np.mean(np.array(theta_fi), axis=0)
fs_var = np.var(np.array(theta_fs), axis=0)
cc_var = np.var(np.array(theta_cc), axis=0)
fi_var = np.var(np.array(theta_fi), axis=0)

# print("Full sample estimator mean: ", fs_mean)
# print("Complete case estimator mean: ", cc_mean)
# print("Fractional Imputation estimator mean: ", fi_mean)
# print()
# print("Full sample estimator variance: ", fs_var)
# print("Complete case estimator variance: ", cc_var)
# print("Fractional Imputation estimator variance: ", fi_var)

In [ ]:
# Import module
from tabulate import tabulate
 
# Assign data
mydata = [
    ["Full Sample", fs_mean, fs_var],
    ["Complete Case", cc_mean, cc_var],
    # ["Fractional Imputation", fi_mean, fi_var],
]
 
# Create header
head = ["Estimator", "Mean", "Variance"]
 
# Display table
print(tabulate(mydata, headers=head, tablefmt="grid"))